In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import PIL
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import FloatVGG16
from SampleNN import *
from tqdm import tqdm_notebook
import torchvision
import SampleVGG16
%matplotlib inline
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [2]:
traindir = '/home/data/yanzy/train'
valdir = '/home/data/yanzy/val'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

val_dataset =  torchvision.datasets.ImageFolder(
    valdir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
]))
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64,
                                         shuffle=False, num_workers = 32, pin_memory=False)

train_dataset =  torchvision.datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
]))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16,
                                         shuffle=True, num_workers = 2, pin_memory=False)

In [3]:
def val(Sample = 1, N = 16, m = 6):
    TMP = protectStateDict(sample_net)
    sampleStateDict(sample_net, N, m)
    sample_net.eval()
    correct = 0
    total = 0
    count = 0
    with torch.no_grad():
        for data in tqdm_notebook(val_loader, desc = "m = %d"%(m), leave = False):
            if count % Sample == 0:
            
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                images = oSample()(images)
                outputs = sample_net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)


                correct += (predicted == labels).sum().item()
                #print correct, total
            count += 1

    print('When m = %d, Accuracy of the network on the %d test images: %.3f %%' % (m, total,
        100.0 * correct / total))
    
    sample_net.load_state_dict(TMP)
    del TMP
    return 100.0 * correct / total

In [4]:
float_net = torchvision.models.vgg16_bn(pretrained=True).to(device)
gt  = float_net.state_dict()
b = gt.keys()
N = 16
for m in range(8,9):
    sample_net = SampleVGG16.SampleVGG16(N = N, m = m, bn = True).to(device)
    lol = sample_net.state_dict()
    a = lol.keys()
    for i in range(len(a)):
        lol[a[i]] = gt[b[i]]
    sample_net.load_state_dict(lol)
    val(Sample = 1, N = N, m = m)

When m = 8, Accuracy of the network on the 50000 test images: 48.424 %


In [7]:
N = 16
m = 6
float_net = torchvision.models.vgg16_bn(pretrained=True)
sample_net = SampleVGG16.SampleVGG16(N = N, m = m, bn = True)
gt  = float_net.state_dict()
b = gt.keys()
lol = sample_net.state_dict()
a = lol.keys()
for i in range(len(a)):
    lol[a[i]] = gt[b[i]]

sample_net.load_state_dict(lol)
sample_net = sample_net.to(device)
val(Sample = 1, N = N, m = m)

When m = 6, Accuracy of the network on the 50000 test images: 0.048 %


0.048

In [5]:
criterion_s = nn.CrossEntropyLoss()
#optimizer_s = optim.SGD(sample_net.parameters(), lr=1e-4, momentum=0.9)
optimizer_s = optim.Adam(sample_net.parameters(), lr=1e-5, weight_decay=1e-6)
def train(epoch, Early_stop = 10000):
    sample_net.train()
    running_loss = 0.0
    count = 0
    for i, data in tqdm_notebook(enumerate(train_loader, 0),desc = "epoch = %d"%(epoch)):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer_s.zero_grad()

        # forward + backward + optimize
        TMP = protectStateDict(sample_net)
        sampleStateDict(sample_net)
        outputs = sample_net(inputs)
        loss = criterion_s(outputs, labels)
        loss.backward()
        sample_net.load_state_dict(TMP)
        torch.cuda.empty_cache()
        optimizer_s.step()
        count += labels.size(0)
        if count >= Early_stop:
            break

        # print statistics
        running_loss += loss.item()
    print('[%d, %5d] loss: %.3f' %
          (epoch + 1, i + 1, running_loss / 1000))
    return running_loss

In [6]:
for epoch in range(4):

    torch.cuda.empty_cache()
    train(epoch, Early_stop = 10000000)
    torch.cuda.empty_cache()
    val(Sample = 1, N = N, m = m)

Process Process-34:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
Process Process-33:
  File "/home/yanzy/.virtualenvs/fp/local/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/queues.py", line 131, in get
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    if timeout < 0 or not self._poll(timeout):
    self.run()
KeyboardInterrupt
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yanzy/.virtualenvs/fp/local/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    s

KeyboardInterrupt: 

In [2]:
inputs = torch.Tensor([1]).requires_grad_()
w = torch.Tensor([10]).requires_grad_()
for i in range(20):
    b = mSample()(w * inputs)
    l = abs(b - 5)
    l.backward()
    w.data = w.data - 0.5 * w.grad.data
    w.grad.zero_()
    print b

tensor([10.], grad_fn=<mSample_F>)
tensor([9.5000], grad_fn=<mSample_F>)
tensor([9.], grad_fn=<mSample_F>)
tensor([8.5000], grad_fn=<mSample_F>)
tensor([8.], grad_fn=<mSample_F>)
tensor([7.5000], grad_fn=<mSample_F>)
tensor([7.], grad_fn=<mSample_F>)
tensor([6.5000], grad_fn=<mSample_F>)
tensor([6.], grad_fn=<mSample_F>)
tensor([5.5000], grad_fn=<mSample_F>)
tensor([5.], grad_fn=<mSample_F>)
tensor([5.], grad_fn=<mSample_F>)
tensor([5.], grad_fn=<mSample_F>)
tensor([5.], grad_fn=<mSample_F>)
tensor([5.], grad_fn=<mSample_F>)
tensor([5.], grad_fn=<mSample_F>)
tensor([5.], grad_fn=<mSample_F>)
tensor([5.], grad_fn=<mSample_F>)
tensor([5.], grad_fn=<mSample_F>)
tensor([5.], grad_fn=<mSample_F>)


In [1]:
from SampleNN import *
print mSample()

mSample(N = 16, m = 6)


In [20]:
print w.requires_grad

True


In [5]:
b = torch.Tensor([1.3]).requires_grad_()
c = b.round()
print c
c.backward()
print b.grad

tensor([1.], grad_fn=<RoundBackward>)
tensor([0.])
